# Self-Driving Car Engineer Nanodegree


## Project: **Finding Lane Lines on the Road** 
***
In this project, you will use the tools you learned about in the lesson to identify lane lines on the road.  You can develop your pipeline on a series of individual images, and later apply the result to a video stream (really just a series of images). Check out the video clip "raw-lines-example.mp4" (also contained in this repository) to see what the output should look like after using the helper functions below. 

Once you have a result that looks roughly like "raw-lines-example.mp4", you'll need to get creative and try to average and/or extrapolate the line segments you've detected to map out the full extent of the lane lines.  You can see an example of the result you're going for in the video "P1_example.mp4".  Ultimately, you would like to draw just one line for the left side of the lane, and one for the right.

In addition to implementing code, there is a brief writeup to complete. The writeup should be completed in a separate file, which can be either a markdown file or a pdf document. There is a [write up template](https://github.com/udacity/CarND-LaneLines-P1/blob/master/writeup_template.md) that can be used to guide the writing process. Completing both the code in the Ipython notebook and the writeup template will cover all of the [rubric points](https://review.udacity.com/#!/rubrics/322/view) for this project.

---
Let's have a look at our first image called 'test_images/solidWhiteRight.jpg'.  Run the 2 cells below (hit Shift-Enter or the "play" button above) to display the image.

**Note: If, at any point, you encounter frozen display windows or other confounding issues, you can always start again with a clean slate by going to the "Kernel" menu above and selecting "Restart & Clear Output".**

---

**The tools you have are color selection, region of interest selection, grayscaling, Gaussian smoothing, Canny Edge Detection and Hough Tranform line detection.  You  are also free to explore and try other techniques that were not presented in the lesson.  Your goal is piece together a pipeline to detect the line segments in the image, then average/extrapolate them and draw them onto the image for display (as below).  Once you have a working pipeline, try it out on the video stream below.**

---

<figure>
 <img src="examples/line-segments-example.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your output should look something like this (above) after detecting line segments using the helper functions below </p> 
 </figcaption>
</figure>
 <p></p> 
<figure>
 <img src="examples/laneLines_thirdPass.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your goal is to connect/average/extrapolate line segments to get output like this</p> 
 </figcaption>
</figure>

**Run the cell below to import some packages.  If you get an `import error` for a package you've already installed, try changing your kernel (select the Kernel menu above --> Change Kernel).  Still have problems?  Try relaunching Jupyter Notebook from the terminal prompt.  Also, consult the forums for more troubleshooting tips.**  

## Import Packages

In [34]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
%matplotlib inline

## Read in an Image

In [2]:
#reading in an image
image = mpimg.imread('test_images/solidWhiteRight.jpg')

#printing out some stats and plotting
print('This image is:', type(image), 'with dimensions:', image.shape)
#plt.imshow(image)  # if you wanted to show a single color channel image called 'gray', for example, call as plt.imshow(gray, cmap='gray')

This image is: <class 'numpy.ndarray'> with dimensions: (540, 960, 3)


## Ideas for Lane Detection Pipeline

**Some OpenCV functions (beyond those introduced in the lesson) that might be useful for this project are:**

`cv2.inRange()` for color selection  
`cv2.fillPoly()` for regions selection  
`cv2.line()` to draw lines on an image given endpoints  
`cv2.addWeighted()` to coadd / overlay two images  
`cv2.cvtColor()` to grayscale or change color  
`cv2.imwrite()` to output images to file  
`cv2.bitwise_and()` to apply a mask to an image

**Check out the OpenCV documentation to learn about these and discover even more awesome functionality!**

## Helper Functions

Below are some helper functions to help get you started. They should look familiar from the lesson!

In [3]:
import math

def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    (assuming your grayscaled image is called 'gray')
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Or use BGR2GRAY if you read an image with cv2.imread()
    # return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    `vertices` should be a numpy array of integer points.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

def average_lines(lines, m, height):
    upper = []
    lower = []
    for line in lines:
        for x1,y1,x2,y2 in line:
            b = y1 - m*x1
            upper.append((height - b) / m)
            lower.append((height/2+50 - b) / m)
                 
    return [int(sum(upper)/len(upper)), int(sum(lower)/len(lower))]


def draw_lines(img, lines, color=[0, 255, 0], thickness=4):
    """
    NOTE: this is the function you might want to use as a starting point once you want to 
    average/extrapolate the line segments you detect to map out the full
    extent of the lane (going from the result shown in raw-lines-example.mp4
    to that shown in P1_example.mp4).  
    
    Think about things like separating line segments by their 
    slope ((y2-y1)/(x2-x1)) to decide which segments are part of the left
    line vs. the right line.  Then, you can average the position of each of 
    the lines and extrapolate to the top and bottom of the lane.
    
    This function draws `lines` with `color` and `thickness`.    
    Lines are drawn on the image inplace (mutates the image).
    If you want to make the lines semi-transparent, think about combining
    this function with the weighted_img() function below
    """
    height, width, _ = img.shape
    left_lane = []
    right_lane = []
    left_slopes = []
    right_slopes = []
    upper_threshold = 0.8
    lower_threshold = 0.3
    for line in lines:
        for x1,y1,x2,y2 in line:
            slope = (y2-y1)/(x2-x1)
            if slope <  -1 * lower_threshold and slope > -1 * upper_threshold:
                left_lane.append(line)
                left_slopes.append(slope)
            elif slope > lower_threshold and slope < upper_threshold:
                right_lane.append(line)
                right_slopes.append(slope)
    
    left_upper_average, left_lower_average = average_lines(left_lane, sum(left_slopes)/len(left_slopes), height)
    right_upper_average, right_lower_average = average_lines(right_lane, sum(right_slopes)/len(right_slopes), height)      
    cv2.line(img, (left_upper_average, height), (left_lower_average, int(height/2+50)), color, thickness)
    cv2.line(img, (right_upper_average, height), (right_lower_average, int(height/2+50)), color, thickness)

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.
        
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    draw_lines(line_img, lines)
    return line_img

# Python 3 has support for cool math symbols.

def weighted_img(img, initial_img, α=0.8, β=1., γ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + γ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, γ)

## Test Images

Build your pipeline to work on the images in the directory "test_images"  
**You should make sure your pipeline works well on these images before you try the videos.**

In [8]:
import os
os.listdir("test_images/")

['whiteCarLaneSwitch.jpg',
 'solidYellowLeft.jpg',
 'solidWhiteRight.jpg',
 'solidWhiteCurve.jpg',
 'solidYellowCurve2.jpg',
 'solidYellowCurve.jpg']

## Build a Lane Finding Pipeline



Build the pipeline and run your solution on all test_images. Make copies into the `test_images_output` directory, and you can use the images in your writeup report.

Try tuning the various parameters, especially the low and high Canny thresholds as well as the Hough lines parameters.

In [9]:
def lane_finding_pipeline(image):
    height, width, _ = image.shape
    grey_image = grayscale(image)

    blurred_image = gaussian_blur(grey_image, 5)

    canny_image = canny(blurred_image, 50, 150)

    bottom_left = (0,height)
    upper_left = (width/2, height/2 + 45)
    upper_right = (width/2, height/2 + 45)
    bottom_right = (width,height)
    vertices = np.array([[bottom_left,upper_left, upper_right, bottom_right]], dtype=np.int32)

    region_of_interest_image = region_of_interest(canny_image, vertices)

    hough_image = hough_lines(region_of_interest_image, 1, np.pi/180, 10, 10, 4)

    weighted_image = weighted_img(hough_image, image)
    
    return weighted_image


image_list = os.listdir("test_images/")

for i in range(len(image_list)):
    image = mpimg.imread("test_images/" + str(image_list[i]))
    processed_image = lane_finding_pipeline(image)
    mpimg.imsave("test_images_output/" + str(image_list[i]), processed_image)

## Test on Videos

You know what's cooler than drawing lanes over images? Drawing lanes over video!

We can test our solution on two provided videos:

`solidWhiteRight.mp4`

`solidYellowLeft.mp4`

**Note: if you get an import error when you run the next cell, try changing your kernel (select the Kernel menu above --> Change Kernel). Still have problems? Try relaunching Jupyter Notebook from the terminal prompt. Also, consult the forums for more troubleshooting tips.**

**If you get an error that looks like this:**
```
NeedDownloadError: Need ffmpeg exe. 
You can download it by calling: 
imageio.plugins.ffmpeg.download()
```
**Follow the instructions in the error message and check out [this forum post](https://discussions.udacity.com/t/project-error-of-test-on-videos/274082) for more troubleshooting tips across operating systems.**

In [28]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [50]:
def process_image(image):
    # NOTE: The output you return should be a color image (3 channel) for processing video below
    # TODO: put your pipeline here,
    # you should return the final output (image where lines are drawn on lanes)
    
    return lane_finding_pipeline(image)

Let's try the one with the solid white lane on the right first ...

In [60]:
white_output = 'test_videos_output/solidWhiteRight.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4").subclip(0,5)
clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)


                                                              
t:  84%|████████▍ | 186/221 [1:08:18<00:00, 88.85it/s, now=None]
                                                                
t:  84%|████████▍ | 186/221 [1:08:18<00:00, 88.85it/s, now=None]

t:   0%|          | 0/221 [00:00<?, ?it/s, now=None]

t:   6%|▋         | 14/221 [00:00<00:01, 138.42it/s, now=None]

Moviepy - Building video test_videos_output/solidWhiteRight.mp4.
Moviepy - Writing video test_videos_output/solidWhiteRight.mp4





t:  13%|█▎        | 29/221 [00:00<00:01, 141.35it/s, now=None]

t:  20%|█▉        | 44/221 [00:00<00:01, 143.77it/s, now=None]

t:  24%|██▍       | 54/221 [00:00<00:01, 120.91it/s, now=None]

t:  29%|██▉       | 64/221 [00:00<00:01, 104.91it/s, now=None]

t:  33%|███▎      | 73/221 [00:00<00:01, 97.78it/s, now=None] 

t:  38%|███▊      | 84/221 [00:00<00:01, 99.74it/s, now=None]

t:  43%|████▎     | 94/221 [00:00<00:01, 97.74it/s, now=None]

t:  47%|████▋     | 104/221 [00:00<00:01, 98.23it/s, now=None]

t:  52%|█████▏    | 115/221 [00:01<00:01, 98.92it/s, now=None]

t:  57%|█████▋    | 125/221 [00:01<00:00, 96.78it/s, now=None]

t:  61%|██████    | 135/221 [00:01<00:00, 95.26it/s, now=None]

t:  66%|██████▌   | 145/221 [00:01<00:00, 92.40it/s, now=None]

t:  70%|███████   | 155/221 [00:01<00:00, 80.78it/s, now=None]

t:  74%|███████▍  | 164/221 [00:01<00:00, 81.55it/s, now=None]

t:  78%|███████▊  | 173/221 [00:01<00:00, 80.25it/s, now=None]

t:  82%|████████▏ | 182/221 [00:01<00:00

Moviepy - Done !
Moviepy - video ready test_videos_output/solidWhiteRight.mp4
CPU times: user 7.18 s, sys: 291 ms, total: 7.47 s
Wall time: 2.74 s


Play the video inline, or if you prefer find the video in your filesystem (should be in the same directory) and play it in your video player of choice.

In [61]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

## Improve the draw_lines() function

**At this point, if you were successful with making the pipeline and tuning parameters, you probably have the Hough line segments drawn onto the road, but what about identifying the full extent of the lane and marking it clearly as in the example video (P1_example.mp4)?  Think about defining a line to run the full length of the visible lane based on the line segments you identified with the Hough Transform. As mentioned previously, try to average and/or extrapolate the line segments you've detected to map out the full extent of the lane lines. You can see an example of the result you're going for in the video "P1_example.mp4".**

**Go back and modify your draw_lines function accordingly and try re-running your pipeline. The new output should draw a single, solid line over the left lane line and a single, solid line over the right lane line. The lines should start from the bottom of the image and extend out to the top of the region of interest.**

Now for the one with the solid yellow lane on the left. This one's more tricky!

In [62]:
yellow_output = 'test_videos_output/solidYellowLeft.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip2 = VideoFileClip('test_videos/solidYellowLeft.mp4').subclip(0,5)
clip2 = VideoFileClip('test_videos/solidYellowLeft.mp4')
yellow_clip = clip2.fl_image(process_image)
%time yellow_clip.write_videofile(yellow_output, audio=False)


                                                                
t:  84%|████████▍ | 186/221 [1:08:44<00:00, 88.85it/s, now=None]
                                                                
t:  84%|████████▍ | 186/221 [1:08:44<00:00, 88.85it/s, now=None]

t:   0%|          | 0/681 [00:00<?, ?it/s, now=None]

t:   2%|▏         | 15/681 [00:00<00:04, 145.05it/s, now=None]

Moviepy - Building video test_videos_output/solidYellowLeft.mp4.
Moviepy - Writing video test_videos_output/solidYellowLeft.mp4





t:   5%|▍         | 31/681 [00:00<00:04, 146.24it/s, now=None]

t:   7%|▋         | 46/681 [00:00<00:04, 146.04it/s, now=None]

t:   8%|▊         | 56/681 [00:00<00:05, 121.46it/s, now=None]

t:  10%|▉         | 66/681 [00:00<00:05, 105.66it/s, now=None]

t:  11%|█         | 75/681 [00:00<00:06, 99.37it/s, now=None] 

t:  12%|█▏        | 84/681 [00:00<00:06, 96.21it/s, now=None]

t:  14%|█▎        | 93/681 [00:00<00:06, 89.10it/s, now=None]

t:  15%|█▌        | 103/681 [00:00<00:06, 90.16it/s, now=None]

t:  16%|█▋        | 112/681 [00:01<00:06, 84.46it/s, now=None]

t:  18%|█▊        | 121/681 [00:01<00:06, 81.75it/s, now=None]

t:  19%|█▉        | 130/681 [00:01<00:06, 79.81it/s, now=None]

t:  20%|██        | 138/681 [00:01<00:07, 77.51it/s, now=None]

t:  21%|██▏       | 146/681 [00:01<00:06, 77.76it/s, now=None]

t:  23%|██▎       | 156/681 [00:01<00:06, 81.75it/s, now=None]

t:  24%|██▍       | 165/681 [00:01<00:06, 83.83it/s, now=None]

t:  26%|██▌       | 174/681 [00:01<00:06

Moviepy - Done !
Moviepy - video ready test_videos_output/solidYellowLeft.mp4
CPU times: user 24.5 s, sys: 826 ms, total: 25.3 s
Wall time: 8.49 s


In [33]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(yellow_output))

## Writeup and Submission

If you're satisfied with your video outputs, it's time to make the report writeup in a pdf or markdown file. Once you have this Ipython notebook ready along with the writeup, it's time to submit for review! Here is a [link](https://github.com/udacity/CarND-LaneLines-P1/blob/master/writeup_template.md) to the writeup template file.


## Optional Challenge

Try your lane finding pipeline on the video below.  Does it still work?  Can you figure out a way to make it more robust?  If you're up for the challenge, modify your pipeline so it works with this video and submit it along with the rest of your project!

In [35]:
challenge_output = 'test_videos_output/challenge.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip3 = VideoFileClip('test_videos/challenge.mp4').subclip(0,5)
clip3 = VideoFileClip('test_videos/challenge.mp4')
challenge_clip = clip3.fl_image(process_image)
%time challenge_clip.write_videofile(challenge_output, audio=False)

NameError: name 'VideoFileClip' is not defined

In [65]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(challenge_output))